In [1]:
import os
import pandas as pd

def exportar_pacientes_para_modeled(diretorio_curated, diretorio_modeled, arquivo_saida, chunksize):
    arquivos_parquet = [f for f in os.listdir(diretorio_curated) if f.startswith('lote_') and f.endswith('.parquet')]
    
    if not arquivos_parquet:
        print("🚨 Nenhum arquivo Parquet encontrado no diretório.")
        return
    
    os.makedirs(diretorio_modeled, exist_ok=True)
    
    lista_pacientes = []

    for arquivo in arquivos_parquet:
        caminho_arquivo = os.path.join(diretorio_curated, arquivo)
        print(f"🔄 Lendo arquivo: {caminho_arquivo}")

        # ✅ Lê o arquivo inteiro (sem chunksize)
        df = pd.read_parquet(caminho_arquivo, engine='pyarrow')

        # ✅ Divide em chunks manualmente
        total_registros = len(df)
        for i in range(0, total_registros, chunksize):
            chunk = df.iloc[i:i + chunksize].reset_index(drop=True)

            # ✅ Filtrar as entradas onde o resourceType é 'Patient'
            df_pacientes = chunk[chunk['resource_resourceType'] == 'Patient']
            
            # ✅ Agrupar por sexo e contar o número de pacientes
            contagem_por_sexo = (df_pacientes
                                 .groupby('resource_gender')
                                 .size()
                                 .reset_index(name='contagem'))
            
            # ✅ Filtrar apenas pacientes do sexo masculino
            contagem_masculino = contagem_por_sexo[contagem_por_sexo['resource_gender'] == 'male']

            lista_pacientes.append(contagem_masculino)

    # ✅ Concatena todos os lotes
    if lista_pacientes:
        df_final = pd.concat(lista_pacientes, ignore_index=True)
        
        # ✅ Consolidar a contagem final de pacientes masculinos
        df_final = (df_final
                    .groupby('resource_gender', as_index=False)
                    .sum()
                    .reset_index(drop=True))
        
        # ✅ Salva como Excel
        caminho_saida = os.path.join(diretorio_modeled, arquivo_saida)
        df_final.to_excel(caminho_saida, index=False)
        print(f"✅ Arquivo de saída salvo em: {caminho_saida}")
    else:
        print("🚨 Nenhum dado de 'Patient' encontrado para exportar.")

# 🔥 Exemplo de uso:
diretorio_curated = r'C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient'
diretorio_modeled = r'C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\modeled'
arquivo_saida = 'contagem_pacientes_masculinos.xlsx'
chunksize = 1000

# ✅ Executa o processo de exportação
exportar_pacientes_para_modeled(diretorio_curated, diretorio_modeled, arquivo_saida, chunksize)


🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient\lote_001.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient\lote_002.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient\lote_003.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient\lote_004.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient\lote_005.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient\lote_006.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient\lote_007.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient\lote_008.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Docu